In [39]:
!pwd

/home/santiago/Clases/semestre-9/disruptica/self-driving-car/experiments/imitation-learning


In [1]:
import pandas as pd

df2 = pd.read_csv('data_test/annotation/obj.csv')
df2.filename = df2.filename.apply(lambda x: 'data_test/images/' + x[2:])

In [2]:
import os
annotated = sorted(os.listdir('data_test/images/'))[:-1]

In [3]:
import pandas as pd
df = pd.DataFrame()
for i in sorted(os.listdir('data_test/annotation/dir')):
    tmp = pd.read_csv('data_test/annotation/dir/' + i, header=None)
    tmp.columns = ['filename', 'dir']
    tmp['filename'] = tmp['filename'].apply(
        lambda x: 'data_test/images/' + i.split('_')[0] + '_' + str(x) + '.jpg')
    df = pd.concat([df, tmp])

In [4]:
from tqdm import tqdm
df2['dir'] = 0
for i in tqdm(range(len(df))):
    df2['dir'][df2.filename == df.iloc[i].filename] = df.iloc[i].dir

  0%|          | 0/3497 [00:00<?, ?it/s]/home/santiago/anaconda3/envs/vehicle/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/santiago/anaconda3/envs/vehicle/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
  6%|▋         | 227/3497 [00:06<01:27, 37.31it/s]


KeyboardInterrupt: 

In [14]:
df2.columns = ['filename', 'width', 'height', 'name', 'xmin', 'ymin', 'xmax', 'ymax', 'dir']

In [15]:
import cv2

In [16]:
magic = []
for i in set(df2.filename):
    try:
        cv2.imread(i).shape
        tmp = df2[df2.filename == i]
        #tmp.columns = ['filename', 'width', 'height', 'name', 'ymin', 'ymax', 'xmin', 'xmax', 'dir']
        magic.append(
            {'object': list(tmp[['name', 'ymin', 'ymax', 'xmin', 'xmax']].T.to_dict().values()),
             'filename': i,
             'height': list(tmp.height)[0],
             'width': list(tmp.width)[0],
             'dir': list(tmp.dir)[0]}
        )
    except:
        print(i)

data_test/images/29_d.jpg
data_test/images/32_d.jpg
data_test/images/34_d.jpg


In [1]:
import pickle

ann = pickle.load(open('annotation.pkl', 'rb'))

In [4]:
from collections import Counter
w = Counter()

In [9]:
d = {k: sum(w.values())/v for k, v in w.items()}

In [14]:
from sklearn.utils import class_weight
import numpy as np
y_train = np.array(list(map(lambda x: x['dir'], ann)))
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [17]:
dict(zip(np.unique(y_train), class_weights))

{' backward': 30.857142857142858,
 ' backward_left': 3.0,
 ' backward_right': 2.7870967741935484,
 ' forward': 0.582995951417004,
 ' forward_left': 0.5503184713375796,
 ' forward_right': 0.5737051792828686}

array([' backward', ' backward_left', ' backward_right', ' forward',
       ' forward_left', ' forward_right'], dtype='<U15')